In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "13119987562885657024", 2, "feature")

In [4]:
emptyStruct = Lar.Struct()

LinearAlgebraicRepresentation.Struct(Any[], Any, "15500481575592296054", Any, "feature")

## Vecchia funzione

In [5]:
function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	for i=1:length(obj.body)
		if isa(obj.body[i],Matrix)
			mat = obj.body[i]
			d,d = size(mat)
			newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
			for h in range(1, length=d)
				for k in range(1, length=d)
					newMat[h,k]=mat[h,k]
				end
			end
			push!(cloned.body,[newMat])
		elseif (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==3
			V,FV,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,FV,EV)])
		elseif  (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==2
			V,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,EV)])
		elseif isa(obj.body[i],Lar.Struct)
			newObj = Struct()
			newObj.box = [ [obj.body[i].box[1];zeros(dimadd)],
							[obj.body[i].box[2];zeros(dimadd)] ]
			newObj.category = (obj.body[i]).category
			push!(cloned.body,[embedTraversal(newObj,obj.body[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 1 method)

In [6]:
@btime embedTraversal(emptyStruct,x,1,"New")

  4.600 μs (20 allocations: 1.53 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]  …  [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 

## Nuove funzioni

In [7]:
function embedTraversalMatrix(objBody)
	mat = objBody[i]
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	for h in range(1, length=d)
		for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 1 method)

In [8]:
function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

In [9]:
function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody[i].box[1];zeros(dimadd)],
				   [objBody[i].box[2];zeros(dimadd)] ]
	newObj.category = (objBody[i]).category
	return
end

embedTraversalStruct (generic function with 1 method)

In [10]:
function embedTraversal(cloned,obj,n,suffix)
	objBody = obj.body
	for i=1:length(objBody)
		if isa(objBody[i],Matrix)
			newMat = embedTraversalMatrix(objBody)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody)
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [11]:
@btime embedTraversal(emptyStruct,x,1,"New")

  4.433 μs (20 allocations: 1.53 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]  …  [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [([0.0 

In [12]:
@benchmark embedTraversal(emptyStruct,x,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  1.53 KiB
  allocs estimate:  20
  --------------
  minimum time:     4.633 μs (0.00% GC)
  median time:      9.625 μs (0.00% GC)
  mean time:        13.566 μs (0.00% GC)
  maximum time:     3.413 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6

In [13]:
@code_warntype embedTraversal(emptyStruct,x,1,"New")

Variables
  #self#::Core.Compiler.Const(embedTraversal, false)
  cloned::LinearAlgebraicRepresentation.Struct
  obj::LinearAlgebraicRepresentation.Struct
  n::Int64
  suffix::String
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  @_8::Any
  @_9::Any
  @_10::Any
  mat::Any
  d::Any
  newMat::Array{Float64,2}
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  V::Any
  FV::Any
  EV::Any
  dimadd::Int64
  ncols::Any
  nmat::Array{Float64,2}
  newObj::Any
  h::Int64
  @_23::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_25::Bool
  @_26::Any
  @_27::Bool
  @_28::Any

Body::LinearAlgebraicRepresentation.Struct
1 ── %1   = Base.getproperty(obj, :body)::Array
│    %2   = Main.length(%1)::Int64
│    %3   = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│           (@_6 = Base.iterate(%3))
│    %5   = (@_6 === nothing)::Bool
│    %6   = Base.not_int(%5)::Bool
└───        goto #30 if not %6
2 ┄─        Core.NewvarNode(:(@_8))
│           Core.N

18 ─ %123 = Base.getproperty(obj, :body)::Array
│    %124 = Base.getindex(%123, i)::Any
│    %125 = (%124 isa Main.Tuple)::Bool
└───        goto #20 if not %125
19 ─        (@_27 = %125)
└───        goto #21
20 ─ %129 = Base.getproperty(obj, :body)::Array
│    %130 = Base.getindex(%129, i)::Any
└───        (@_27 = %130 isa Main.Array)
21 ┄        goto #23 if not @_27
22 ─ %133 = Base.getproperty(obj, :body)::Array
│    %134 = Base.getindex(%133, i)::Any
│    %135 = Main.length(%134)::Any
│           (@_28 = %135 == 2)
└───        goto #24
23 ─        (@_28 = false)
24 ┄        goto #26 if not @_28
25 ─ %140 = Base.getproperty(obj, :body)::Array
│    %141 = Base.getindex(%140, i)::Any
│    %142 = Main.deepcopy(%141)::Any
│    %143 = Base.indexed_iterate(%142, 1)::Any
│           (V = Core.getfield(%143, 1))
│           (@_10 = Core.getfield(%143, 2))
│    %146 = Base.indexed_iterate(%142, 2, @_10)::Any
│           (EV = Core.getfield(%146, 1))
│           (dimadd = n)
│           (ncols

## Modifiche